In [1]:
!python --version

Python 3.11.12


In [2]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: unsloth 2025.3.19
Uninstalling unsloth-2025.3.19:
  Successfully uninstalled unsloth-2025.3.19
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-b49n1zse/unsloth_f40e351f7a174368a8eac5cf6ae6820f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-b49n1zse/unsloth_f40e351f7a174368a8eac5cf6ae6820f
  Resolved https://github.com/unslothai/unsloth.git to commit f6f5e6397acfb5737dcb2062a72e860cc3657f6f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.19-py3-none-any.whl size=192490 sha256=bbacd22a5eba68c464a756a1ac0379f68e3a9d83c5a520725b960a276451c6d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-_8ldpzht/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth


In [4]:
!pip install evaluate rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3297417087ea53648a37703900799e85d59eff03bbf5219e4f59121ca744e478
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [5]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported

# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

<ipython-input-5-47bd71ed1ebe>:4: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth.chat_templates import get_chat_template


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from transformers import AutoTokenizer
from unsloth import FastLanguageModel

# 保存したモデルのパス。
# model_path = "/content/drive/MyDrive/ColabNotebooks/Llama_3_2_3B_Instruct_bnb_4bit_personal_info_masking_model"
model_path = "自身のローカルのパス/Llama_3_2_3B_Instruct_bnb_4bit_personal_info_masking_model"

# モデルとトークナイザーの読み込み
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=2048,
    load_in_4bit=True,
    dtype=None,
)

# 推論用に設定
model = model.eval()

# 以下は推論の例
def generate_masked_text(input_text):
    # プロンプトテンプレート
    prompt = f"""
# 個人情報マスキングタスク

あなたは個人情報のマスキングを行うAIアシスタントです。日本語テキスト内の個人情報のみを正確にマスキングしてください。

## マスキング対象の個人情報と形式
- 人名: <マスキング済みの氏名>
- 会社名: <マスキング済みの会社名>
- 住所: <マスキング済みの住所>
- メールアドレス: <マスキング済みのemailアドレス>
- 電話番号: <マスキング済みの電話番号>
- 生年月日: <マスキング済みの生年月日>
- 郵便番号: <マスキング済みの郵便番号>

## 制約条件
- before_maskの個人情報以外の部分は一切変更しないでください。
- before_maskのテキストの構造と文脈を完全に保持して、個人情報のみをマスキングし、after_maskに出力すること
- マスキング後の出力はbefore_maskのテキストの構造と文脈を完全に保持して個人情報のみをマスキングしafter_maskに出力すること

### before_mask:
{input_text}

### after_mask:
"""

    # トークナイズしてモデルに入力
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 生成
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=2048,
            temperature=0.1,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True,
        )

    # 生成されたテキストをデコード
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # "after_mask:" より後の部分を抽出
    try:
        masked_text = generated_text.split("### after_mask:")[1].strip()
    except IndexError:
        # 適切な形式で出力されなかった場合
        masked_text = "出力形式エラー: " + generated_text

    return masked_text

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [7]:
test_text = """
山田太郎様
住所: 東京都渋谷区神南1-2-3 アップルマンション501
電話: 03-1234-5678
メール: yamada.taro@example.com
株式会社テックソリューション
"""

result = generate_masked_text(test_text)
print("元のテキスト:")
print(test_text)
print("\nマスキング後:")
print(result)

元のテキスト:

山田太郎様
住所: 東京都渋谷区神南1-2-3 アップルマンション501
電話: 03-1234-5678
メール: yamada.taro@example.com
株式会社テックソリューション


マスキング後:
<マスキング済みの氏名>様
住所: <マスキング済みの住所> アップルマンション501
電話: <マスキング済みの電話番号>
メール: <マスキング済みのemailアドレス>
<マスキング済みの会社名>


In [8]:
test_text = """
こんにちは。私の名前は佐藤一郎で、株式会社 SoftBank Japan でエンジニアとして働いています。サーバーやネットワークに関するプロジェクトを多く担当しており、最近は特にデータベースの最適化やセキュリティについて深く学んでいます。仕事の関係で、東京の新宿区に住んでいますが、愛知県の出身です。生まれた年月日は1985年の5月3日です。休日はハイキングや読書を楽しんでおり、特に歴史小説が好きです。また、私のメールアドレスは ichiro.sato@example.com で、緊急の場合は、070-9876-5432 までお電話いただければすぐに対応可能です。この業界では常に新しい技術が生まれるため、自己学習を続けることが重要だと思っています。近々、海外のカンファレンスにも参加する予定で、そこでのネットワーキングを通じて、さらなるスキルアップを目指しています。日本の技術者として誇りを持って仕事ができることを嬉しく思っています。
"""

result = generate_masked_text(test_text)
print("元のテキスト:")
print(test_text)
print("\nマスキング後:")
print(result)

元のテキスト:

こんにちは。私の名前は佐藤一郎で、株式会社 SoftBank Japan でエンジニアとして働いています。サーバーやネットワークに関するプロジェクトを多く担当しており、最近は特にデータベースの最適化やセキュリティについて深く学んでいます。仕事の関係で、東京の新宿区に住んでいますが、愛知県の出身です。生まれた年月日は1985年の5月3日です。休日はハイキングや読書を楽しんでおり、特に歴史小説が好きです。また、私のメールアドレスは ichiro.sato@example.com で、緊急の場合は、070-9876-5432 までお電話いただければすぐに対応可能です。この業界では常に新しい技術が生まれるため、自己学習を続けることが重要だと思っています。近々、海外のカンファレンスにも参加する予定で、そこでのネットワーキングを通じて、さらなるスキルアップを目指しています。日本の技術者として誇りを持って仕事ができることを嬉しく思っています。


マスキング後:
こんにちは。私の名前は<マスキング済みの氏名>で、<マスキング済みの会社名> でエンジニアとして働いています。サーバーやネットワークに関するプロジェクトを多く担当しており、最近は特にデータベースの最適化やセキュリティについて深く学んでいます。仕事の関係で、<マスキング済みの住所>に住んでいますが、愛知県の出身です。生まれた年月日は<マスキング済みの生年月日>です。休日はハイキングや読書を楽しんでおり、特に歴史小説が好きです。また、私のメールアドレスは <マスキング済みのemailアドレス> で、緊急の場合は、<マスキング済みの電話番号> にお電話いただければすぐに対応可能です。この業界では常に新しい技術が生まれるため、自己学習を続けることが重要だと思っています。近々、海外のカンファレンスにも参加する予定で、そこでのネットワーキングを通じて、さらなるスキルアップを目指しています。日本の技術者として誇りを持って仕事ができることを嬉しく思っています。
